In [4]:
## Pacotes necessários:
#install.packages("e1071") 
#install.packages("caret")
#install.packages("randomForest")
library("caret")
library("randomForest")

In [7]:
#leitura dos dados

dados <- read.csv("databases/4 - Banco - Dados.csv")

head(dados)

,age,job,marital,education,default,balance,housing,loan,y
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
1,30,unemployed,married,primary,no,1787,no,no,no
2,33,services,married,secondary,no,4789,yes,yes,no
3,35,management,single,tertiary,no,1350,yes,no,no
4,30,management,married,tertiary,no,1476,yes,yes,no
5,59,bluecollar,married,secondary,no,0,yes,no,no
6,35,management,single,tertiary,no,747,no,no,no


In [8]:
#separação da base em treino e teste
set.seed(2034)
indices <- createDataPartition(dados$y, p = 0.80, list = FALSE)
treino <- dados[indices, ]
teste <- dados[-indices, ]


In [11]:
#gerando modelo com random forest
set.seed(2034)
rf <- train(y ~ ., data = treino, method = "rf")
rf
#predicao com o arquivo de teste
predict_rf <- predict(rf, teste)
confusionMatrix(predict_rf, as.factor(teste$y))

Random Forest 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 241, 241, 241, 241, 241, 241, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa     
   2    0.8714131  0.02574122
  11    0.8484603  0.11672744
  21    0.8308457  0.10522291

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   7
       yes  0   0
                                          
               Accuracy : 0.8814          
                 95% CI : (0.7707, 0.9509)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.59891         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.8814          
         Neg Pred Value :    NaN          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : no              
                                 

In [12]:
#aplicando cross-validation
ctrl <- trainControl(method = "cv", number = 10)

set.seed(2034)
rf <- train(y ~ ., data =  treino, method = "rf", trControl = ctrl)
rf

#experimentando na base de teste
predict_rf <- predict(rf, teste)
confusionMatrix(predict_rf, as.factor(teste$y))

Random Forest 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 217, 217, 216, 217, 217, 217, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa       
   2    0.8673333  -0.006666667
  11    0.8678333   0.192994849
  21    0.8596667   0.183174603

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 11.

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  51   7
       yes  1   0
                                          
               Accuracy : 0.8644          
                 95% CI : (0.7502, 0.9396)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.7378          
                                          
                  Kappa : -0.0306         
                                          
 Mcnemar's Test P-Value : 0.0771          
                                          
            Sensitivity : 0.9808          
            Specificity : 0.0000          
         Pos Pred Value : 0.8793          
         Neg Pred Value : 0.0000          
             Prevalence : 0.8814          
         Detection Rate : 0.8644          
   Detection Prevalence : 0.9831          
      Balanced Accuracy : 0.4904          
                                          
       'Positive' Class : no              
                                 

In [14]:
#experimentando com outros valores de mtry
tune_grid <- expand.grid(mtry = c(2, 5, 7, 9))

set.seed(2034)
rf <- train(y ~ ., data = treino, method = "rf", trControl = ctrl, tuneGrid = tune_grid)
rf

#experimentando novo modelo na base de teste
predict_rf <- predict(rf, teste)
confusionMatrix(predict_rf, as.factor(teste$y))

Random Forest 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 217, 217, 216, 217, 217, 217, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa       
  2     0.8673333  -0.006666667
  5     0.8798333   0.166666667
  7     0.8758333   0.159829060
  9     0.8841667   0.253162393

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 9.

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  51   7
       yes  1   0
                                          
               Accuracy : 0.8644          
                 95% CI : (0.7502, 0.9396)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.7378          
                                          
                  Kappa : -0.0306         
                                          
 Mcnemar's Test P-Value : 0.0771          
                                          
            Sensitivity : 0.9808          
            Specificity : 0.0000          
         Pos Pred Value : 0.8793          
         Neg Pred Value : 0.0000          
             Prevalence : 0.8814          
         Detection Rate : 0.8644          
   Detection Prevalence : 0.9831          
      Balanced Accuracy : 0.4904          
                                          
       'Positive' Class : no              
                                 

In [16]:
# predicao de novos casos
dados_novos_casos <- read.csv("databases/4 - Banco - Novos Casos.csv")
View(dados_novos_casos)
dados_novos_casos$y <- NULL

predict_rf <- predict(rf, dados_novos_casos)
resultado <- cbind(dados_novos_casos, predict_rf)
View(resultado)

age,job,marital,education,default,balance,housing,loan,y
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
60,unemployed,married,primary,no,2000,yes,yes,?
33,services,married,secondary,yes,3000,yes,no,?
15,management,single,tertiary,no,1350,yes,no,?


age,job,marital,education,default,balance,housing,loan,predict_rf
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<fct>
60,unemployed,married,primary,no,2000,yes,yes,no
33,services,married,secondary,yes,3000,yes,no,no
15,management,single,tertiary,no,1350,yes,no,no
